In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train = pd.read_pickle(FILE.train_final.value)
# train_shiyi = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
# test_shiyi = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')[['instance_id']].copy()
test = pd.read_pickle(FILE.test_final.value)

# train = train_shiyi.merge(train,how='inner',on='instance_id').reset_index(drop=True)
# test = test_shiyi.merge(test,how='inner',on='instance_id').reset_index(drop=True)
data = pd.concat([train, test],sort=False)

In [3]:
ad_feats = ["adid", "orderid","advert_industry_inner","advert_id","creative_id","campaign_id"]
user_feats = ["model","city","osv","carrier","nnt"]
app_feats = ["app_cate_id", "f_channel", "app_id", "inner_slot_id"]

In [4]:
cv = CountVectorizer(min_df=1)
cv.fit(data['user_tags'].astype(str))
one_hot_tag = cv.transform(data['user_tags'].astype(str))
print(one_hot_tag.shape)

(3072915, 1417)


In [89]:
col2_name = 'model'
alreadyEncode = False
print(data[col2_name].nunique())
if not alreadyEncode:
    le = LabelEncoder()
    data[col2_name] = le.fit_transform(data[col2_name].astype(str))
    alreadyEncode=True



20331


In [90]:
row_index = []
col_index = []
col2_values = data[col2_name].values
for i in tqdm(range(len(data))):

    utags = list(one_hot_tag[i,:].indices)
    col_values = [col2_values[i] for j in range(len(utags))]
    row_index.extend(utags)
    col_index.extend(col_values)
    

100%|██████████| 3072915/3072915 [03:57<00:00, 12916.44it/s]


In [91]:
row_index2 = []
col_index2 = []
col2_values = data[col2_name].values
target_values = data.click.values
for i in tqdm(range(len(data))):
    if target_values[i] != 1:
        continue
    utags = list(one_hot_tag[i,:].indices)
    col_values = [col2_values[i] for j in range(len(utags))]
    row_index2.extend(utags)
    col_index2.extend(col_values)
    

100%|██████████| 3072915/3072915 [00:40<00:00, 75073.81it/s]


In [92]:
assert len(row_index) == len(col_index)
ones = np.ones(len(row_index))
rating = sparse.coo_matrix((ones, (row_index, col_index)))

# rating = (rating > 0) * 1.0

In [93]:
rating = rating.tocsr()
print(rating.shape)
print(len(rating.indices)/rating.shape[0]/rating.shape[1])

(1417, 20332)
0.09121397087806075


In [94]:
assert len(row_index) == len(col_index)
ones = np.ones(len(row_index2))
rating2 = sparse.coo_matrix((ones, (row_index2, col_index2)))

rating2 = rating2.tocsr()
print(rating2.shape)
print(len(rating2.indices)/rating2.shape[0]/rating2.shape[1])

s_append=sparse.csr_matrix((rating2.shape[0],rating.shape[1]-rating2.shape[1]))
rating2_append = sparse.hstack([rating2,s_append],format='csr')
print(rating2_append.shape)

row_list3 = []
col_list3 = []
value_list3 = []
for row in tqdm(range(rating2_append.shape[0])):
    s = rating2_append[row,:]
    for col in s.indices:
        row_list3.append(row)
        col_list3.append(col)
        value_list3.append(rating2_append[row,col]/rating[row,col])
        
rating_new = sparse.coo_matrix((value_list3,(row_list3,col_list3)))
print(rating_new.shape)

  0%|          | 4/1417 [00:00<00:38, 36.81it/s]

(1417, 20326)
0.03127191909489992
(1417, 20332)


100%|██████████| 1417/1417 [00:54<00:00, 25.79it/s]


(1417, 20326)


In [95]:
from scipy import sparse
from scipy.sparse.linalg import svds
n_component=36
# [u, s, vt] = svds(rating, k=n_component)
[u, s, vt] = svds(rating_new, k=n_component)
print(u.shape)
print(s.shape)
print(vt.shape)
print(s[::-1])
col1_col2 = np.diag(s[::-1])

(1417, 36)
(36,)
(36, 20326)
[217.9231563   83.2129715   65.7218494   49.1339183   45.77918365
  40.94592637  39.38969226  38.07682772  35.65369284  33.95848113
  32.63059251  31.30363404  30.93403697  29.85485371  28.4011992
  27.85259242  27.59667653  27.14507364  26.83881763  26.23950063
  25.652358    25.44304079  25.35149403  24.93550809  24.63835609
  24.4848962   23.75729558  23.5019843   23.31960204  22.83342968
  22.29073145  22.21789375  21.95010513  21.61136206  21.35971412
  21.17839387]


In [96]:
u_sort = u[:,::-1]
tags = one_hot_tag.dot(u_sort)

In [97]:
# u_sort = u[:,::-1]
# u_sort = u_sort[:,0]
# tags = one_hot_tag.multiply(u_sort)
# print(tags.shape)

# from scipy.sparse import csr_matrix
# def save_csr(matrix,file_prefix='csr'):
#     data_file = file_prefix + '_data.npy'
#     indices_file = file_prefix + '_indices.npy'
#     indptr_file = file_prefix + '_indptr.npy'
#     np.save(data_file,matrix.data)
#     print('data save')
#     np.save(indices_file,matrix.indices)
#     print('indices save')
#     np.save(indptr_file,matrix.indptr)
#     print('indptr save')
#     print('save done!')
# tags = csr_matrix(tags)
# tags_train = tags[:len(train),:]
# tags_test = tags[len(train):,:]
# save_csr(tags,file_prefix='../../data/features/user_tags/tag_adid')
# # save_csr(tags_train,file_prefix='../../data/features/user_tags/train_tag_adid')
# # save_csr(tags_test,file_prefix='../../data/features/user_tags/test_tag_adid')


In [98]:
col1_topics = pd.DataFrame(tags)
col1_name = 'utagsIndividual'
col1_topics.columns = ['{}_{}_svd_{}'.format(col1_name, col2_name, i) for i in range(n_component)]
col1_embed = col1_topics.values

df_save = data.copy()
for col in col1_topics:
    df_save[col] = col1_topics[col].values

In [99]:
col2_topics = pd.DataFrame(vt.transpose()[:, ::-1])
col2_topics.columns = ['{}_{}_svd_{}'.format(col2_name, col1_name, i) for i in range(n_component)]
col2_embed = col2_topics.values
col2_cnt = len(col2_topics)
col2_topics[col2_name] = range(col2_cnt)

In [100]:
df_save = df_save.merge(col2_topics, on=col2_name, how='left')

In [101]:
save_col = ['instance_id']
for col in df_save:
    if 'svd' in col:
        save_col.append(col)

In [102]:
dff_save = df_save[save_col].copy()
dff_save.to_pickle('../../data/features/agg/svd_utag_model_mean.pkl')

In [53]:
dff_save.shape

(3072915, 33)

In [47]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [48]:
data['user_tags_encoded'] = le.fit_transform(data['user_tags'].astype(str))

In [49]:
def compute_the_id(row, joinby='-'):
    the_str = ''
    for val in row.values:
        the_str += joinby+str(val)
#         if val == 95764:  95764 => user tags is NAN
#             return np.nan 
    return the_str[1:]

In [50]:
user_id_cols = ['city','osv','make','model','user_tags_encoded']
data['user_id'] = data[user_id_cols].apply(lambda row: compute_the_id(row) , axis=1)

# Codes

In [51]:
from scipy import sparse
from scipy.sparse.linalg import svds

def svd_two_cols(df, col1_name, col2_name, n_component):
    data = np.ones(len(df))
    le = LabelEncoder()
    df = df.copy()
    df[col1_name] = le.fit_transform(df[col1_name].astype(str))
    df[col2_name] = le.fit_transform(df[col2_name].astype(str))
    col1 = df[col1_name].values
    col2 = df[col2_name].values

    col1_cnt = max(col1)+1
    col2_cnt = max(col2)+1
    print(col1_cnt, col2_cnt)

    rating = sparse.coo_matrix((data, (col1, col2)))
    rating = (rating > 0) * 1.0

    [u, s, vt] = svds(rating, k=n_component)
    print(u.shape)
    print(s.shape)
    print(vt.shape)
    print(s[::-1])
    col1_col2 = np.diag(s[::-1])

    col1_topics = pd.DataFrame(u[:, ::-1])
    col1_topics.columns = ['{}_{}_svd_{}'.format(col1_name, col2_name, i) for i in range(n_component)]
    col1_embed = col1_topics.values
    col1_topics[col1_name] = range(col1_cnt)

    df = df.merge(col1_topics, on=col1_name, how='left')

    col2_topics = pd.DataFrame(vt.transpose()[:, ::-1])
    col2_topics.columns = ['{}_{}_svd_{}'.format(col2_name, col1_name, i) for i in range(n_component)]
    col2_embed = col2_topics.values
    col2_topics[col2_name] = range(col2_cnt)

    df = df.merge(col2_topics, on=col2_name, how='left')

    return df, col1_col2, col1_embed, col2_embed

In [52]:
df, user_ad_for_dot, user_ad_user_embed, user_ad_ad_embed = svd_two_cols(data, 'user_id', 'adid', 16)
df, model_ad_for_dot, model_ad_model_embed, model_ad_ad_embed = svd_two_cols(df, 'model', 'adid', 16)
df, tag_ad_for_dot, tag_ad_tag_embed, tag_ad_ad_embed = svd_two_cols(df, 'user_tags_encoded', 'adid', 16)

2327425 2221
(2327425, 16)
(16,)
(16, 2221)
[350.12076538 345.73541756 340.3279445  301.66167836 297.86039802
 289.18351978 272.05388218 261.50472411 252.33731351 225.78723232
 220.6932322  216.59175075 215.55702342 198.29491748 192.99452896
 187.70081697]
20332 2221
(20332, 16)
(16,)
(16, 2221)
[333.47204852 147.80008126 105.99190644  94.08657677  87.33754918
  81.82397571  74.31152966  66.63746345  54.68527316  53.30165307
  48.79600707  48.35081725  43.01864511  41.91258192  39.3089441
  38.75585021]
1345254 2221
(1345254, 16)
(16,)
(16, 2221)
[286.05661523 249.19831531 238.67673108 234.67312317 224.08247975
 213.68687174 203.34021612 201.84383008 193.92289273 179.12998691
 172.16582183 167.21125061 158.79903506 151.83018445 150.17206646
 142.12058069]


In [54]:
df_save = df[['instance_id']].copy()
for col in df:
    if 'svd' in col:
        df_save[col] = df[col].values

In [55]:
df_save.to_pickle('../../data/features/agg/svd_raw.pkl')

In [32]:
df = data.copy()
col1_name = 'user_id'
col2_name = 'adid'
data = np.ones(len(df))
le = LabelEncoder()
df = df.copy()
df[col1_name] = le.fit_transform(df[col1_name].astype(str))
df[col2_name] = le.fit_transform(df[col2_name].astype(str))
col1 = df[col1_name].values
col2 = df[col2_name].values


In [36]:
col1_cnt = max(col1)+1
col2_cnt = max(col2)+1
print(col1_cnt, col2_cnt)
rating = sparse.lil_matrix((col1_cnt, col2_cnt))
for v1,v2 in tqdm(zip(col1,col2)):
    rating[v1,v2] += 1

2327425 2221


3072915it [00:18, 166142.01it/s]


In [38]:
rating = sparse.coo_matrix(rating)
n_component = 32
[u, s, vt] = svds(rating, k=n_component)
print(u.shape)
print(s.shape)
print(vt.shape)

(2327425, 32)
(32,)
(32, 2221)


In [8]:
df, user_ad_for_dot, user_ad_user_embed, user_ad_ad_embed = svd_two_cols(df, 'user_id', 'adid', 32)
df, model_ad_for_dot, model_ad_model_embed, model_ad_ad_embed = svd_two_cols(df, 'model', 'adid', 32)
df, tag_ad_for_dot, tag_ad_tag_embed, tag_ad_ad_embed = svd_two_cols(df, 'user_tags_sorted', 'adid', 32)
print(df.shape)

NameError: name 'df' is not defined